In [1]:
import numpy as np
from matplotlib import pyplot as plt

from calmutils.stitching.fusion import fuse_image_blockwise
from calmutils.stitching.stitching import stitch
from calmutils.misc.json import recursive_dict_query
from autosted.data import HDF5DataReader
from autosted.utils.parameter_constants import OFFSET_STAGE_GLOBAL_PARAMETERS, PIXEL_SIZE_PARAMETERS

In [2]:
h5file = '/Volumes/cooperation_data/ArgyrisPapantonis-nuclear_architecture/David_Hoerl/auto_sir_comp_analysis_workspace/raw_data_compressed_h5/20210326_IMR90_young_untr/rep4/b40a6115aae453d1abdc6b90a18b56dc.h5'

reader = HDF5DataReader(h5file)

In [ ]:
# pick images to fuse
# NOTE: should have the same pixel size, do not mix overview-detail
# idxs_to_fuse = [(i,) for i in range(9)]
idxs_to_fuse = [i for i in reader.keys() if len(i) == 1]

imgs_to_fuse = []
translation_vectors = []
first_translation_vec = None

for idx in idxs_to_fuse:
    
    # get image data, drop empty time dimension
    img = reader[idx].data[0][0].squeeze()
    imgs_to_fuse.append(img.astype(float))
    
    # get stage position, make translation matrix
    settings = reader[idx].measurement_settings[0]
    stage_coords = [recursive_dict_query(settings, key) for key in OFFSET_STAGE_GLOBAL_PARAMETERS]
    pixel_sizes = [recursive_dict_query(settings, key) for key in PIXEL_SIZE_PARAMETERS]
    # NOTE: we flip z position, so stage agrees with image coordinates
    translation_vec = np.array([-1, 1, 1]) * np.array(stage_coords) / np.array(pixel_sizes)
    if first_translation_vec is None:
        first_translation_vec = translation_vec
    translation_vectors.append(translation_vec - first_translation_vec)
    
transforms = stitch(imgs_to_fuse, translation_vectors, corr_thresh=0.75)

# run fusion and show
img_fused = fuse_image_blockwise(imgs_to_fuse, transforms, block_size=[25, 1500, 1500], interpolation_mode='linear')
plt.imshow(img_fused.max(axis=0), cmap='magma')

In [ ]:
# pick center of images to fuse
only_earlier_idxs = True
center_idx = (40,)
idxs_to_fuse = []

# get stage position of center tile
settings = reader[center_idx].measurement_settings[0]
stage_coords_center = [recursive_dict_query(settings, key) for key in OFFSET_STAGE_GLOBAL_PARAMETERS]

# collect tiles within range of center
for idx in reader.keys():
    
    if len(idx) != 1:
        continue
    
    # get stage position
    settings = reader[idx].measurement_settings[0]
    stage_coords = [recursive_dict_query(settings, key) for key in OFFSET_STAGE_GLOBAL_PARAMETERS]
    
    if (np.linalg.norm(np.array(stage_coords_center) - np.array(stage_coords)) < 75e-6
        and (not only_earlier_idxs or idx[0] <= center_idx[0])):
        idxs_to_fuse.append(idx)
    
print('stitching indices:', idxs_to_fuse)

imgs_to_fuse = []
translation_vectors = []

for idx in idxs_to_fuse:
    
    # get image data, drop empty time dimension
    img = reader[idx].data[0][0].squeeze()
    imgs_to_fuse.append(img)
    
    # get stage position, make translation matrix
    settings = reader[idx].measurement_settings[0]
    stage_coords = [recursive_dict_query(settings, key) for key in OFFSET_STAGE_GLOBAL_PARAMETERS]
    pixel_sizes = [recursive_dict_query(settings, key) for key in PIXEL_SIZE_PARAMETERS]
    # NOTE: we flip z position, so stage agrees with image coordinates
    tr = np.array([-1, 1, 1]) * np.array(stage_coords) / np.array(pixel_sizes)
    translation_vectors.append(tr)
    
transforms = stitch(imgs_to_fuse, translation_vectors, subpixel=True, corr_thresh=0.75, reference_idx=0)

# run fusion and show
img_fused = fuse_image_blockwise(imgs_to_fuse, transforms, block_size=[25, 1500, 1500])
plt.imshow(img_fused.max(axis=0), cmap='magma')

In [4]:
import napari

if napari.current_viewer() is not None:
    napari.current_viewer().close()
    
viewer = napari.view_image(img_fused, scale=(3,1,1))

In [ ]:
from calmutils.imageio.tiff_imagej import save_tiff_imagej

save_tiff_imagej('/Users/david/Desktop/test.tif', img_fused.astype(np.uint16), axes='zyx')